###General Instructions
Complete the following Python script per the instructions provided at the top of each code block. Weblogs from the smu.edu website from Oct 25 through Nov 3, 2018 are provided at *\<your filestore path\>*/weblogs/new/.  In the following cells, you will briefly examine one of these files:

In [0]:
# notebook config
USER_NAME = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
FILE_STORE_ROOT = '/FileStore/shared_uploads/'+USER_NAME

In [0]:
# list the log files in the folder
display(
  dbutils.fs.ls(FILE_STORE_ROOT + '/weblogs/new')
  )

path,name,size,modificationTime
dbfs:/FileStore/shared_uploads/grilla@smu.edu/weblogs/new/u_ex181025_x.log,u_ex181025_x.log,163718499,1678644239000


In [0]:
# examine the headers and first line of data from one of these files
dbutils.fs.head(FILE_STORE_ROOT + '/weblogs/new/u_ex181025_x.log').split('\r\n')[:5]

[Truncated to first 65536 bytes]
Out[3]: ['#Software: Microsoft Internet Information Services 10.0',
 '#Version: 1.0',
 '#Date: 2018-10-25 00:00:02',
 '#Fields: date time s-ip cs-method cs-uri-stem cs-uri-query s-port cs-username cs(User-Agent) cs(Referer) sc-status sc-substatus sc-win32-status time-taken X-Forwarded-For',
 '2018-10-24 23:59:30 129.119.66.37 GET /-/media/Site/News/NewsSources/EarthquakeStudy/earthquake-causes-17may2016.jpg h=338&la=en&w=350&hash=4865678DB6FEA8BB4AECD9F687D0DE225EAAC3BE 443 - Mozilla/5.0+(compatible;+SeznamBot/3.2;++http://napoveda.seznam.cz/en/seznambot-intro/) - 200 0 0 691 77.75.78.161']

Looking at the file contents, you should notice these logs use a different format than the one we explored in class.  That's because earlier in 2018, SMU made some changes to their webservers that caused them to default to what is known as the [W3C Extended Format](https://www.w3.org/TR/WD-logfile.html), which you can read about here: https://docs.microsoft.com/en-us/windows/desktop/http/w3c-logging.

Come up with a RegEx pattern that captures all the data (in the data lines). Don't worry about excluding the one metadata line that represents the header for your data lines; you can add a filter method call to remove this line later.

When constructing your RegEx pattern, you will want to note that the W3C Extended Format captures data with no spaces in any fields.  All fields are separated with a single space. This should allow you to specify a relatively simple RegEx pattern.

In [0]:
weblog_file_name = FILE_STORE_ROOT + '/weblogs/new/u_ex181025_x.log'

In [0]:
import re

def get_fields(line):
    regex_pattern = r'^(\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)'
    m = re.match(regex_pattern, line)
    if m is not None:
        return m.groups()

# read the raw log data
raw_lines = sc.textFile(weblog_file_name)

# remove any metadata lines 
remove_meta = raw_lines.filter(lambda line: not line.startswith('#'))

# parse log data using get_fields and filter out any poorly parsed lines
parsed = remove_meta.map(get_fields).filter(lambda fields: fields is not None)

parsed.take(1)

Out[5]: [('2018-10-24',
  '23:59:30',
  '129.119.66.37',
  'GET',
  '/-/media/Site/News/NewsSources/EarthquakeStudy/earthquake-causes-17may2016.jpg',
  'h=338&la=en&w=350&hash=4865678DB6FEA8BB4AECD9F687D0DE225EAAC3BE',
  '443',
  '-',
  'Mozilla/5.0+(compatible;+SeznamBot/3.2;++http://napoveda.seznam.cz/en/seznambot-intro/)',
  '-',
  '200',
  '0',
  '0',
  '691',
  '77.75.78.161')]

Verify that less than 1% of lines were removed given previous steps.  If 1% or more lines have been removed, revisit the previous cell, verifying your RegEx pattern and any filtering logic.  Do not proceed until you are below the 1% data loss threshold.

In [0]:
line_count = raw_lines.count()
parsed_count = parsed.count()
percent_diff = (100. * (line_count - parsed_count))/line_count

print('The number of lines in the file is {0}'.format(line_count))
print('The number of lines parsed from the file is {0}'.format(parsed_count))
print('There is a {0:.4f}% difference in element counts between the two rdds'.format(percent_diff))

The number of lines in the file is 571392
The number of lines parsed from the file is 571384
There is a 0.0014% difference in element counts between the two rdds


With your data properly parsed, you now need to convert your date and time fields into a single datetime field.  Use the datetime library to assist you in this.  Remember to filter out any lines that may not parse correctly.  You should expect to lose no lines in this file due to bad date and time values.  Keep in mind that in the W3C Extended format, the date and time values are recorded as separate fields.  You will need to combine them in order to construct a proper datetime value:

In [0]:
from datetime import datetime

# append a datetime field to your rdd, filtering out any poorly parsed datetime values
# be sure to leave your rdd in a state where each element is a tuple consisting of 16 values,
# the last of which is a datetime value
dt_converted = parsed.map(lambda fields: fields + (datetime.strptime(fields[0] + " " + fields[1], '%Y-%m-%d %H:%M:%S'),)).filter(lambda fields: fields[15] is not None)

# verify the structure of your rdd elements
dt_converted.take(1)

Out[7]: [('2018-10-24',
  '23:59:30',
  '129.119.66.37',
  'GET',
  '/-/media/Site/News/NewsSources/EarthquakeStudy/earthquake-causes-17may2016.jpg',
  'h=338&la=en&w=350&hash=4865678DB6FEA8BB4AECD9F687D0DE225EAAC3BE',
  '443',
  '-',
  'Mozilla/5.0+(compatible;+SeznamBot/3.2;++http://napoveda.seznam.cz/en/seznambot-intro/)',
  '-',
  '200',
  '0',
  '0',
  '691',
  '77.75.78.161',
  datetime.datetime(2018, 10, 24, 23, 59, 30))]

Again, we need to verify we loss less than 1% of data given the filter above.  If you exceed this threshold, return to the previous cell and make appropriate corrections to your logic:

In [0]:
dt_count = dt_converted.count()
percent_diff = (100. * (parsed_count - dt_count))/parsed_count

print('The number of parsed lines is {0}'.format(parsed_count))
print('The number of parsed lines with bad datetime values is {0}'.format(dt_count))
print('There is a {0:.4f}% difference in element counts between the two rdds'.format(percent_diff))

The number of parsed lines is 571384
The number of parsed lines with bad datetime values is 571384
There is a 0.0000% difference in element counts between the two rdds


You are just about ready to convert your RDD to a DataFrame, but because you will want your sc-status and time-taken fields to be brought into the data frame as integer values, you'll need to convert those fields to integers now:

In [0]:
# convert the sc-status and time-taken fields to integer values
cleansed = dt_converted.map(lambda fields: (fields[0], fields[1], fields[2], fields[3], fields[4], fields[5], fields[6], fields[7], fields[8], fields[9], int(fields[10]), fields[11], fields[12], int(fields[13]), fields[14], fields[15]))


cleansed.take(1)

Out[9]: [('2018-10-24',
  '23:59:30',
  '129.119.66.37',
  'GET',
  '/-/media/Site/News/NewsSources/EarthquakeStudy/earthquake-causes-17may2016.jpg',
  'h=338&la=en&w=350&hash=4865678DB6FEA8BB4AECD9F687D0DE225EAAC3BE',
  '443',
  '-',
  'Mozilla/5.0+(compatible;+SeznamBot/3.2;++http://napoveda.seznam.cz/en/seznambot-intro/)',
  '-',
  200,
  '0',
  '0',
  691,
  '77.75.78.161',
  datetime.datetime(2018, 10, 24, 23, 59, 30))]

You know have a well-formed data set within your RDD.  Define a schema for this data set and convert it, using that schema, into a data frame.  Once converted to a dataframe, drop the date and time fields but keep the datetime value which you created in previous cells. Also, drop the sc_substatus and sc_win32_status fields as they are not needed.

When defining your schema, be sure to use field names corresponding to those in the header line of your log file.  Be sure to replace all dashes in those field names with underscores; the dashes make for some confusing SQL syntax later.

Also, be sure to type all fields as strings except for sc_status and time_taken which should be integers and the utc_time field (which will be mapped to the datetime value constructed in the last cell) which should be typed as a timestamp:

In [0]:
from pyspark.sql.types import *

my_schema = StructType([
  StructField('date', StringType()),
  StructField('time', StringType()),
  StructField('s_ip', StringType()),
  StructField('cs_method', StringType()),
  StructField('cs_uri_stem', StringType()),
  StructField('cs_uri_query', StringType()),
  StructField('s_port', StringType()),
  StructField('cs_username', StringType()),
  StructField('cs_user_agent', StringType()),
  StructField('cs_referer', StringType()),
  StructField('sc_status', IntegerType()), 
  StructField('sc_substatus', StringType()), 
  StructField('sc_win32_status', StringType()),
  StructField('time_taken', IntegerType()), 
  StructField('X_Forwarded_For', StringType()),
  StructField('utc_time', TimestampType()) 
  ])

# create a dataframe
df = spark.createDataFrame( cleansed, schema=my_schema)

# drop fields as instructed above
focused = df.drop('date','time','sc_substatus','sc_win32_status')

# display some results to verify code
focused.show()

+-------------+---------+--------------------+--------------------+------+-----------+--------------------+--------------------+---------+----------+---------------+-------------------+
|         s_ip|cs_method|         cs_uri_stem|        cs_uri_query|s_port|cs_username|       cs_user_agent|          cs_referer|sc_status|time_taken|X_Forwarded_For|           utc_time|
+-------------+---------+--------------------+--------------------+------+-----------+--------------------+--------------------+---------+----------+---------------+-------------------+
|129.119.66.37|      GET|/-/media/Site/New...|h=338&la=en&w=350...|   443|          -|Mozilla/5.0+(comp...|                   -|      200|       691|   77.75.78.161|2018-10-24 23:59:30|
|129.119.66.37|      GET|                   /|                   -|   443|          -|Mozilla/5.0+(iPho...|                   -|      200|       187| 98.156.209.143|2018-10-24 23:59:31|
|129.119.66.37|      GET|/-/media/Site/Mai...|                   -|   

Now that we have a DataFrame, register it as a temporary view named *logs*.

In [0]:
focused.createOrReplaceTempView("logs")

spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |     logs|       true|
+--------+---------+-----------+



In our log data set, the cs_uri_stem represents the asset requested from the web server.  This will include a mix of web pages, images, and scripts. Write a query that returns all fields from logs where where cs-uri-stem **DOES NOT** include the following image file types:
* png files
* jpeg & jpg files
* gif files
* js files
* css files
* ico files
* pdf files
* ashx files

Save this as a temporary view named *pages*.

In [0]:
query = '''
SELECT *
FROM logs
WHERE 
  cs_uri_stem NOT LIKE '%.png' AND
  cs_uri_stem NOT LIKE '%.jpeg' AND
  cs_uri_stem NOT LIKE '%.jpg' AND
  cs_uri_stem NOT LIKE '%.gif' AND
  cs_uri_stem NOT LIKE '%.js' AND
  cs_uri_stem NOT LIKE '%.css' AND
  cs_uri_stem NOT LIKE '%.ico' AND
  cs_uri_stem NOT LIKE '%.pdf' AND
  cs_uri_stem NOT LIKE '%.ashx'
'''

# execute the query, capturing results to a temporary view
spark.sql(query).createOrReplaceTempView("pages")

spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |     logs|       true|
|        |    pages|       true|
+--------+---------+-----------+



Write a query that answers the question, what are the most frequently visited pages accessed from the default smu.edu web page?  Keep in mind that the default smu.edu web page has the cs_referer string of *https://www.smu.edu/*. Use the show method to display the first 20 results to the screen, sorted from most frequent to less frequent:

In [0]:
query = '''
SELECT
  cs_uri_stem as Page,
  count(*) as PageViews
FROM pages
WHERE cs_referer = 'https://www.smu.edu/'
GROUP BY cs_uri_stem
ORDER BY  PageViews DESC
LIMIT 20
'''

spark.sql(query).show(truncate=False)

+-----------------------------------------------------+---------+
|Page                                                 |PageViews|
+-----------------------------------------------------+---------+
|/Admission/Academics/Majors/MajorsGrid               |150      |
|/admission                                           |115      |
|/Admission/Academics/Majors                          |107      |
|/AboutSMU                                            |69       |
|/Admission/CampusLife                                |69       |
|/                                                    |67       |
|/BusinessFinance/HR/WorkingatSMU                     |57       |
|/Admission/Apply                                     |43       |
|/AboutSMU/Administration                             |36       |
|/cox                                                 |32       |
|/Graduate                                            |31       |
|/academics                                           |28       |
|/dedman  